In [1]:
import sys
import json
import argparse

parser = argparse.ArgumentParser("simple_example")
from helm.benchmark.runner import InstanceGenerations,GenerationSummary
from process_gens import get_gen_summary, get_run_folder

import ProcessGenMetrics
from helm.common.general import ensure_directory_exists, write, asdict_without_nones
import os



parser.add_argument("--model", help="model used for task", type=str)
parser.add_argument("--base_folder", help="model used for task", type=str)
parser.add_argument("--eval_instances", help="number of instances to eval", type=str)
parser.add_argument("--task_name", help="task name", type=str)
parser.add_argument("--num_beams", help="number of beams", type=str)
parser.add_argument("--metric_names", help="metrics in question", type=str)

args = parser.parse_args()

base_folder=args.base_folder
num_beams=args.num_beams
model=args.model
task_name=args.task_name
eval_instances=args.eval_instances
metric_names=args.metric_names


# num_beams=32
# model="meta_llama_Llama_3.1_8B_Instruct"
# eval_instances=5
# task_name="wmt_14_language_pair_de_en_"
# root_folder=f"snellius_copies/wmt_test"
# base_folder=f"{root_folder}/eval_{eval_instances}"
# metric_names="test"

metric_names_list=metric_names.split(",")
metrics=ProcessGenMetrics.get_metrics(metric_names_list)

generation_summary=get_gen_summary(base_folder, num_beams, model, task_name, eval_instances)

for instance_generation in generation_summary.instance_generations:
    for idx,generated_output in enumerate(instance_generation.examples):
        pd_metrics_dict= {} if generated_output.stats_dict is None else generated_output.stats_dict
        for metric in metrics:
            pd_metrics_dict=ProcessGenMetrics.calculate_post_metric(pd_metrics_dict,metric,instance_generation,generated_output)
        generated_output.stats_dict=pd_metrics_dict



/opt/miniconda3/envs/crfm-helm2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


run_path=get_run_folder(base_folder, num_beams, model, task_name, eval_instances)
write(
    os.path.join(run_path, "generation_summary_metrics.json"),
    json.dumps(asdict_without_nones(generation_summary),indent=2)
)
metric_path=os.path.join(run_path, "generation_summary_metrics.json")
print(metric_path)

0
Writing 555414 characters to snellius_copies/wmt_test/eval_5/wmt_14_language_pair_de_en_/meta_llama_Llama_3.1_8B_Instruct/32_beams/runs/eval_5/generation_summary_metrics.json
snellius_copies/wmt_test/eval_5/wmt_14_language_pair_de_en_/meta_llama_Llama_3.1_8B_Instruct/32_beams/runs/eval_5/generation_summary_metrics.json
